In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import os
import sys
sys.path.append('../utils')
import databalancing

/home/pricie/frederic/name_clustering_transformer/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
! nvidia-smi

Tue Sep 30 12:14:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:01:00.0 Off |                  N/A |
| 34%   37C    P8             36W /  350W |       1MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
rebalance = True
experiment_name = "bert-finetune-name-similarity"
if rebalance:
    experiment_name+='_rebalanced'
experiment_name

'bert-finetune-name-similarity_rebalanced'

In [4]:
gpu_device = 0 #use gpu 0
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)


In [5]:
# 1. Load the dataset
data = pd.read_csv(r"../data/pairdata.csv")

In [6]:
# 2. Split into train, validation, test
train_df = data[data["dataset"] == "train"]
val_df = data[data["dataset"] == "validation"]
test_df = data[data["dataset"] == "test"]
mixed_df = data[data["dataset"] == "mixed"] 

#TODO: catch cases where data is mixed!

In [7]:
if rebalance: 
    train_df = databalancing.rebalance_dataset(train_df)
    val_df = databalancing.rebalance_dataset(val_df)
    test_df = databalancing.rebalance_dataset(test_df)

In [8]:
print(len(data), len(train_df), len(val_df), len(test_df), len(mixed_df))

346715 156150 51924 100640 128188


In [9]:
# 3. Dataset class
class NamePairDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=32):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text1, text2, label = str(row.name1), str(row.name2), int(row.are_same)
        encoding = self.tokenizer(
            text1,
            text2,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [10]:
# 4. Tokenizer and datasets
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_dataset = NamePairDataset(train_df, tokenizer)
val_dataset = NamePairDataset(val_df, tokenizer)
test_dataset = NamePairDataset(test_df, tokenizer)

In [11]:
# 5. Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# 6. Metrics function
def ez_metrics(labels, preds):
    return {
            'accuracy': accuracy_score(labels, preds),
            'precision': precision_score(labels, preds),
            'recall': recall_score(labels, preds),
            'f1': f1_score(labels, preds)
        }


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return ez_metrics(labels, preds)
    

In [13]:
# 7. Training arguments
training_args = TrainingArguments(
    output_dir="./results_"+experiment_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [14]:
# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_1933537/3871163508.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# 9. Train
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.095200,0.463948,0.895289,0.891206,0.900508,0.895833
2,0.048200,0.711165,0.871081,0.938627,0.794084,0.860326
3,0.018700,0.759799,0.888163,0.920088,0.850166,0.883746
4,0.016000,0.824116,0.883927,0.934333,0.825899,0.876776
5,0.003900,1.047649,0.880537,0.940557,0.812418,0.871804


TrainOutput(global_step=48800, training_loss=0.05389170395668413, metrics={'train_runtime': 2131.2143, 'train_samples_per_second': 366.34, 'train_steps_per_second': 22.898, 'total_flos': 1.283899727952e+16, 'train_loss': 0.05389170395668413, 'epoch': 5.0})

In [16]:
# 10. Baseline evaluation on test with random guessing: 
actuals = test_df['are_same']
preds = torch.randint(low=0, high=2, size=(len(actuals),))
print("Random guessing baseline on test set:")
ez_metrics(actuals, preds)

Random guessing baseline on test set:


{'accuracy': 0.49839030206677265,
 'precision': 0.4983926020003175,
 'recall': 0.49910572337042924,
 'f1': 0.4987489077766304}

In [17]:
# 10. Evaluate on test set
metrics = trainer.evaluate(test_dataset)
print("Test metrics:", metrics)

Test metrics: {'eval_loss': 0.6525376439094543, 'eval_accuracy': 0.8580087440381559, 'eval_precision': 0.8455253366066976, 'eval_recall': 0.8760731319554849, 'eval_f1': 0.8605282164399071, 'eval_runtime': 77.1492, 'eval_samples_per_second': 1304.486, 'eval_steps_per_second': 81.53, 'epoch': 5.0}


In [18]:
# 11. Save model
trainer.save_model(f"../completed_experiments/{experiment_name}")
tokenizer.save_pretrained(f"../completed_experiments/{experiment_name}")

('../completed_experiments/bert-finetune-name-similarity_rebalanced/tokenizer_config.json',
 '../completed_experiments/bert-finetune-name-similarity_rebalanced/special_tokens_map.json',
 '../completed_experiments/bert-finetune-name-similarity_rebalanced/vocab.txt',
 '../completed_experiments/bert-finetune-name-similarity_rebalanced/added_tokens.json')